# Probar Web API ya creada en forma pública para utilizar modelo entrenado con TensorFlow+Keras usando Flask y ngrok

In [ ]:
#@title Configuración

ngrok_public_Web_API = "https://aff3-34-74-225-24.ngrok-free.app/iris" #@param{type:"string"}



In [ ]:
#@title Probar Web API con un ejemplo

import requests
import json

LargoSepalo = 5.5 #@param{type:"number"}
AnchoSepalo = 2.6 #@param{type:"number"}
LargoPetalo = 4.4 #@param{type:"number"}
AnchoPetalo = 1.2 #@param{type:"number"}

dictValues = {
    "LargoSepalo" : LargoSepalo,
    "AnchoSepalo" : AnchoSepalo,
    "LargoPetalo" : LargoPetalo,
    "AnchoPetalo" : AnchoPetalo
    }

# función auxiliar para probar API
def ejecutarModelWebAPI(dictValues, URL_API):
  # ejecuta la web API usando request
  headers = {"content-type": "application/json",
             "ngrok-skip-browser-warning": "69420" }
  data_json = json.dumps( dictValues )
  json_response = requests.post(URL_API, data=data_json, headers=headers)
  if json_response.status_code == 200:
    return json.loads(json_response.text)
  else:
    print(json_response, ":", json_response.content)
    return "ERROR"

# ejecuta API
print("")
print("> Datos: ", dictValues)
print("> Resultado: ", ejecutarModelWebAPI(dictValues, ngrok_public_Web_API) )
print("")



> Datos:  {'LargoSepalo': 5.5, 'AnchoSepalo': 2.6, 'LargoPetalo': 4.4, 'AnchoPetalo': 1.2}
> Resultado:  {'clase': 'Versicolor', 'claseID': 2}



In [ ]:
#@title Probar Web API con ejemplos de un CSV que tiene los datos mostrados en <construir-RNA-MLP-IRIS.ipynb>
from google.colab import files

separador_valores = ";" #@param{type:"string"}
if separador_valores == "":
  separador_valores = ";"

def cargarArchivo(fn):

  # lo carga en lista
  uploadedData = []
  with open(fn, 'r', encoding='utf-8') as f:
    contents = f.readlines()
  # separa en lineas
  uploadedData.extend( ("\n".join(contents)).split("\n") )

  print('-- Archivo "{name}" con largo {length} bytes cargado'.format(
    name=fn, length=len(uploaded[fn])))

  return uploadedData

# sube archivo
uploaded = files.upload()
# procesa los datos
uploadedData = []
for fn in uploaded.keys():
    uploadedData.extend( cargarArchivo(fn))

if len(uploadedData)>0:
  print("\n")
  # procesa los datos cargados
  classReal = []
  classPreds = []
  for data in uploadedData:
    if data!="":
      arAux = data.split(separador_valores)
      dictValues = {
      "LargoSepalo" : arAux[0],
      "AnchoSepalo" : arAux[1],
      "LargoPetalo" : arAux[2],
      "AnchoPetalo" : arAux[3]
      }
      classReal.append( arAux[4] )
      claseModelo = ejecutarModelWebAPI(dictValues, ngrok_public_Web_API)
      if "clase" in claseModelo:
        classPreds.append( str(claseModelo["claseID"]) )
      else:
        print("Clase no identificada: ", dictValues, "->", claseModelo)
        classPreds.append( "ERROR" )
  print("\n")

  from sklearn.metrics import classification_report
  from sklearn.metrics import confusion_matrix
  import pandas as pd

  # muestra reporte de clasificación
  print("\n Reporte de Clasificación: ")
  print(classification_report(classReal, classPreds))

  # muestra matriz de confusion
  print('\nMatriz de Confusión ( real / modelo ): ')
  cm = confusion_matrix(classReal, classPreds)
  cmtx = pd.DataFrame(
      cm
    )
  # agrega para poder mostrar la matrix de confusión completa
  pd.options.display.max_rows = 100
  pd.options.display.max_columns = 100
  cmtx.sort_index(axis=0, inplace=True)
  cmtx.sort_index(axis=1, inplace=True)
  print(cmtx)
  print("\n")


Saving datos_PRUEBA.csv to datos_PRUEBA.csv
-- Archivo "datos_PRUEBA.csv" con largo 684 bytes cargado





 Reporte de Clasificación: 
              precision    recall  f1-score   support

           1       1.00      1.00      1.00        13
           2       1.00      0.92      0.96        13
           3       0.92      1.00      0.96        12

    accuracy                           0.97        38
   macro avg       0.97      0.97      0.97        38
weighted avg       0.98      0.97      0.97        38


Matriz de Confusión ( real / modelo ): 
    0   1   2
0  13   0   0
1   0  12   1
2   0   0  12


